In [2]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


def gen_gpt_messages(prompt):
    messages = [{"role": "user", "content": prompt}]
    return messages


def get_completion(prompt, model="gpt-3.5-turbo", temperature = 0):
    response = client.chat.completions.create(
        model=model,
        messages=gen_gpt_messages(prompt),
        temperature=temperature,
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"

In [3]:
prod_review_zh = """
这个熊猫公仔是我给女儿的生日礼物，她很喜欢，去哪都带着。
公仔很软，超级可爱，面部表情也很和善。但是相比于价钱来说，
它有点小，我感觉在别的地方用同样的价钱能买到更大的。
快递比预期提前了一天到货，所以在送给女儿之前，我自己玩了会。
"""

文本概括，考虑不同侧重点。
文本内容提取

#### 单一文本概括

In [4]:
prompt = f"""
您的任务是从电子商务网站上生成一个产品评论的简短摘要。

请对三个反引号之间的评论文本进行概括，最多30个词汇。

评论:```{prod_review_zh}```
"""

response = get_completion(prompt)
print(response)

熊猫公仔软可爱，女儿喜欢，面部表情友善。但价钱略贵，尺寸小。快递提前到货，满意。


In [5]:
##侧重快递服务
prompt = f"""
您的任务是从电子商务网站上生成一个产品评论的简短摘要。

请对三个反引号之间的评论文本进行概括，最多30个词汇，并且聚焦在产品运输上。

评论:```{prod_review_zh}```
"""

response = get_completion(prompt)
print(response)

产品可爱软绵，面部表情友好，但稍小。快递提前一天送达。愉快购物体验。


In [6]:
##侧重产品价格和质量
prompt = f"""
您的任务是从电子商务网站上生成一个产品评论的简短摘要。

请对三个反引号之间的评论文本进行概括，最多30个词汇，并且聚焦在产品价格和质量。

评论:```{prod_review_zh}```
"""

response = get_completion(prompt)
print(response)

价格适中，质量可爱，但稍小。快递快速。


#### 关键信息提取

In [7]:
prompt = f"""
您的任务是从电子商务网站的产品评论中提取相关信息。
请从以下三个反引号之间的评论文本中提取产品运输相关信息，最多30个词
评论:```{prod_review_zh}```
"""
response = get_completion(prompt)
print(response)

产品运输相关信息：快递比预期提前了一天到货。


#### 同时概括多条文本

In [9]:
review_1 = prod_review_zh

# review for a standing lamp
review_2 = """
我想为我的卧室找一个漂亮的灯，这款灯还有额外的存储空间，价格也不太高。\
购买后很快就收到了，两天就送到了。但在运输过程中，灯的拉链断了，公司态度\
很好，发来了一条新的。新的拉链也在几天内就到了。这个灯非常容易装配。后来，我\
发现缺少一个部分，所以我联系了他们的客户支持，他们很快就给我寄来了缺失的部件\
！我觉得这是一家非常关心他们的客户和产品的好公司。
"""

# review for an electric toothbrush
review_3 = """
我的牙科卫生师推荐我使用电动牙刷，这就是我购买这款牙刷的原因。目前为止，我发现电池的\
续航时间颇为令人印象深刻。在初次充电并在第一周保持充电器插头插入以调节电池状态之后，我\
已经将充电器拔掉，并在过去的3周里，每天两次刷牙都使用同一次充电。然而，这款牙刷的刷头实\
在太小了。我见过的婴儿牙刷都比这个大。我希望牙刷头能做得更大一些，搭配不同长度的刷毛更好\
地清洁牙齿间缝，因为现有的无法做到这一点。总的来说，如果你能以大约50美元的价格购入这款电动\
牙刷，那它就物超所值。厂家配套的替换刷头价格相当昂贵，但你可以买到价格更为合理的通用款。\
使用这款牙刷让我感觉像每天都去看了牙医一样，我的牙齿感觉洁净如新！
"""

# review for a blender
review_4 = """
他们还在11月把17件套系统以大约$49的优惠价格销售，几乎是五折。但不明原因（轻易就可以归咎为价格欺诈）\
在到了12月第二周，同一套系统的价格一下儿飙升到了$70-$89之间。11件套系统的价格也从之前的优惠价$29上\
升了大概$10。看上去还算公道，但如果你仔细观察底部，会发现刀片锁定的部位相比几年前的版本要略逊一筹，所\
以我打算非常小心翼翼地使用（例如，我会将像豆子、冰块、大米之类的硬质食材先用搅拌机压碎，然后调到我需要\
的份量，再用打发刀片研磨成更细的粉状，制作冰沙时我首选交叉刀片，如果需要更细腻些或者少些浆糊状，我会换成\
平刀）。在制作果昔时，把将要用的水果和蔬菜切片冷冻是个小技巧（如果你打算用菠菜，要先稍微焖炖软，再冷冻，\
制作雪葩时，用一个小到中号的食品加工器就行）这样就不用或者很少加冰块到你的果昔了。大约一年后，电机开始发出\
一些可疑的声音。我联系了客服，但保修期已经过期，所以我只好另购一台。友情提示：这类产品的整体质量都在下滑，\
所以他们更多的是利用品牌知名度和消费者的忠诚度来保持销售。我在两天之后就收到了它。
"""

reviews = [review_1, review_2, review_3, review_4]

In [11]:
for i in range(len(reviews)):
    prompt = f"""
        你的任务是从电子商务网站上的产品评论中提取相关信息。
        请对三个反引号之间的评论文本进行概括，最多20个词汇。
        评论文本:```{reviews[i]}```
    """
    response = get_completion(prompt)
    print(i,response,"\n")

0 熊猫公仔，女儿生日礼物，软可爱，面部表情和善，价钱有点小，快递提前到货。 

1 评论概括：灯具带存储空间，价格合理，快递速度快，售后服务好，易装配，客户支持快速寄送缺失部件，好公司。 

2 评论概括：电动牙刷续航时间长，刷头小，价格合理，清洁效果好，推荐购买。 

3 评论提到产品在不同时间点的价格变化，质量略有下降，使用小心翼翼，制作果昔的小技巧，电机问题，品牌知名度。 

